#### Author: Arun Ramesh, University of Houston. https://rpglab.github.io/people/Arun-Venkatesh-Ramesh/
#### Source webpage: https://rpglab.github.io/resources/FL-ML-R-SCUC_Python/
#### If you use any codes/data here for your work, please cite the following paper: 
#####       Arun Venkatesh Ramesh and Xingpeng Li, “Feasibility Layer Aided Machine Learning Approach for Day-Ahead Operations”, IEEE Transactions on Power Systems, Apr. 2023.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
np.random.seed(1)

import sys
nums = np.arange(2000)
np.set_printoptions(threshold=sys.maxsize)

import csv
import time
from timeit import default_timer as timer

In [3]:
##### data set 24 Bus 8 Prd

dfX_24 = pd.read_csv("demand24Bus24Prd.txt")
dfY_24 = pd.read_csv("commitment24Bus24Prd.txt")

print(dfX_24.info)
print(dfY_24.info)

x = dfX_24.to_numpy()

# normalize X with Base MVA
x = x/100
#print(x[1])

y = dfY_24.to_numpy()
#print(y[1])

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 20% for validation data that we never train over.
split_at = len(x) - len(x) // 5
(x_train, x_test) = x[:split_at], x[split_at:]
(y_train, y_test) = y[:split_at], y[split_at:]


print(len(x_train))
print(len(y_train))

print(len(x_test))
print(len(y_test))

print(indices)

#print(x_train[1])
#print(y_train[1])

#print(x_test[1])
#print(y_test[1])

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

<bound method DataFrame.info of       90.13728903773256  80.95663922833387  75.11440753144379  \
0             67.493145          60.618843          56.244287   
1            107.022171          96.121765          89.185142   
2            108.926408          97.832052          90.772007   
3             62.949194          56.537702          52.457662   
4             95.801737          86.044153          79.834781   
...                 ...                ...                ...   
1494         101.205071          83.534633          79.818072   
1495          84.521178          78.343324          74.534879   
1496          92.821142          84.133780          89.090335   
1497          78.007085          86.473328          79.682256   
1498          85.612332          75.013867          67.089895   

      128.52909733158162  59.25692149702789  123.5214701628187  \
0              96.240225          44.370493          92.490606   
1             152.605688          70.357168         146

In [4]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=300)
neigh.fit(x_train, y_train)

y_hat_tr = neigh.predict(x_train)
y_hat_ts = neigh.predict(x_test)

print (neigh.score(x_train, y_train))
print (neigh.score(x_test, y_test))

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts)) * 100))

# print(y_hat_tr.shape)
# print(y_hat_tr)
# y_hat_ts = neigh.predict(x_test)
# print(y_hat_ts.shape)
# print(y_hat_ts)



0.3416666666666667
0.3177257525083612
train accuracy: 97.47716750841751 %
test accuracy: 97.41900611465829 %


In [5]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

#plot_confusion_matrix(neigh, x_test, y_test)  
#plt.show()
print (y_test.shape)
print (y_hat_ts.shape)
NgxNt = y_test.shape[1]
M = y_test.shape[0]
#print(confusion_matrix(y_test[:,0], y_hat_ts[:,1]))
print (y_test[:,0].shape)
TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

(299, 792)
(299, 792)
(299,)
0.5076602141819533 0.46652984696462957 0.016722408026755852 0.009087530826661262


In [6]:
A_hat_ts = neigh.predict_proba(x_test)
#print(A_hat_ts.shape)
print(A_hat_ts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [108]:
from sklearn.ensemble import RandomForestClassifier
RFclf = RandomForestClassifier(n_estimators = 1, max_depth=2, random_state=0)
RFclf.fit(x_train, y_train)
print (RFclf.score(x_train, y_train))
print (RFclf.score(x_test, y_test))

y_hat_tr = RFclf.predict(x_train)
y_hat_ts = RFclf.predict(x_test)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts)) * 100))

0.30083333333333334
0.26755852842809363
train accuracy: 97.31334175084174 %
test accuracy: 97.28049728049729 %


In [109]:
NgxNt = y_test.shape[1]
M = y_test.shape[0]
#print(confusion_matrix(y_test[:,0], y_hat_ts[:,1]))
print (y_test[:,0].shape)
TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

print ((TP+TN)/(TP+TN+FP+FN))

(299,)
0.5023690078037905 0.4704359650011824 0.012816289990203034 0.014378737204824162
0.9728049728049728


In [14]:
print (y_test[3])
print (y_hat_ts[3])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [19]:
from sklearn.neural_network import MLPClassifier

nfeat = x_train.shape[1]

MlpClf = MLPClassifier(solver='adam', learning_rate='adaptive', alpha=1e-5, hidden_layer_sizes=(nfeat,), activation='relu', random_state=1, max_iter=75, shuffle=False)
MlpClf.fit(x_train,y_train)

print (MlpClf.score(x_train, y_train))
print (MlpClf.score(x_test, y_test))

y_hat_tr_mlp = MlpClf.predict(x_train)
y_hat_ts_mlp = MlpClf.predict(x_test)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr_mlp)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts_mlp)) * 100))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  warnings.warn(


0.3408333333333333
0.31438127090301005
train accuracy: 97.4765361952862 %
test accuracy: 97.45658930441539 %


In [20]:
NgxNt = y_test.shape[1]
M = y_test.shape[0]

TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts_mlp[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

0.5029390898956116 0.4716268031485423 0.011625451842843148 0.01380865511300294


In [22]:
varAccMLP = np.zeros(NgxNt)
for i in range(M):
    for j in range(NgxNt):
        varAccMLP[j] = varAccMLP[j] + (np.abs(y_test[i,j] - y_hat_ts_mlp[i,j])/M*100)
        
for j in range(NgxNt):
    varAccMLP[j] = 100 - varAccMLP[j]
    
print(np.amin(varAccMLP))
print(np.mean(varAccMLP))
print(varAccMLP)

75.25083612040132
97.45658930441539
[100.         100.          91.97324415  92.97658863 100.
 100.          97.32441472  96.98996656 100.         100.
 100.         100.         100.         100.         100.
 100.         100.         100.          99.66555184 100.
  93.31103679  91.97324415 100.         100.         100.
 100.         100.         100.         100.         100.
  97.65886288  95.31772575  96.98996656 100.         100.
  92.30769231  92.97658863 100.          99.66555184  97.65886288
  96.98996656  99.66555184 100.          99.33110368 100.
 100.         100.         100.          99.66555184 100.
  99.33110368  99.33110368 100.          95.65217391  91.97324415
 100.         100.         100.         100.         100.
 100.         100.         100.          97.99331104  96.32107023
  96.98996656 100.          99.66555184  90.96989967  92.64214047
 100.         100.          97.32441472  96.98996656  94.64882943
  96.32107023  97.32441472 100.         100.         1

In [21]:
print (y_test[3])
print (y_hat_ts_mlp[3])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [23]:
print (MlpClf.predict_proba(x_test))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
from sklearn.linear_model import LogisticRegression

TrainScore = []
TrainHardScore = []
TestScore = []
TestHardScore = []
NgxNt = y_train.shape[1]
m_Tr = x_train.shape[0]
m_Ts = x_test.shape[0]

Y_hat_tr_skln_pred = np.zeros((m_Tr,NgxNt))
Y_hat_ts_skln_pred = np.zeros((m_Ts,NgxNt))

start = timer()
for i in range(NgxNt):
    #print ("Ng*Nt: ",i)   
    
    if set([0,1]).issubset(set(y_train[:,i])):      
        
        logreg = LogisticRegression(random_state=0, solver='liblinear', max_iter = 200)
        logreg.fit(x_train,y_train[:,i]) #original shape before transposed
        Y_hat_tr_skln_pred[:,i] = logreg.predict(x_train)
        TrainScore.append(logreg.score(x_train,y_train[:,i]))
        Y_hat_ts_skln_pred[:,i] = logreg.predict(x_test)
        TestScore.append(logreg.score(x_test,y_test[:,i]))
    
    else:
        
        if set([0]).issubset(set(y_train[:,i])):
            Y_hat_tr_skln_pred[:,i] = np.zeros(m_Tr)
        else:
            Y_hat_tr_skln_pred[:,i] = np.ones(m_Tr)
        
        if set([0]).issubset(set(y_test[:,i])):
            Y_hat_ts_skln_pred[:,i] = np.zeros(m_Ts)
        else:
            Y_hat_ts_skln_pred[:,i] = np.ones(m_Ts)
            
        TrainScore.append(1)
        TestScore.append(1)
end = timer()
print("training time: ",end - start)
print ("-------------------------------------------------------")

print("training Score: ", np.mean(TrainScore))
print("testing Score: ", np.mean(TestScore))

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - Y_hat_tr_skln_pred)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - Y_hat_ts_skln_pred)) * 100))

training time:  16.94194480002625
-------------------------------------------------------
training Score:  0.9795717592592592
testing Score:  0.9755413668457147
train accuracy: 97.95717592592592 %
test accuracy: 97.55202526941657 %


In [115]:
M = y_test.shape[0]

TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], Y_hat_ts_skln_pred[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

0.5047295699469613 0.4707906827472045 0.01246157224418094 0.012018175061653322


In [100]:
correctSeq = 0
wrongseq = 0

yDiff = y_test - Y_hat_ts_skln_pred

for i in range(299):
    if -1 < np.sum(yDiff[i]) < 1: 
        correctSeq = correctSeq + 1
    else:
        wrongseq = wrongseq + 1
        
print (correctSeq/299)

0.27424749163879597


In [83]:
A = [1, 2, 3]
B = [1,2,4]
if A==B:
    print("yes")
else: 
    print("no")

no


In [89]:
print (y_test[2])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [99]:
print (yDiff[7])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [121]:
x_test_round = np.around(x_test, decimals=4)

with open("demand24Bus24PrdCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(x_test_round)

In [102]:
with open("commitment_LR_24Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(Y_hat_ts_skln_pred)

In [120]:
with open("commitment_MLP_24Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(y_hat_ts_mlp)

In [112]:
#same variable for KNN and RF. So re-run the classifier and print to correct file
with open("commitment_KNN_24Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(y_hat_ts)

In [110]:
print (0.5023690078037905+0.4704359650011824+0.012816289990203034+0.014378737204824162)

1.0


In [10]:
from sklearn.neural_network import MLPClassifier

nfeat = x_train.shape[1]

MlpClfLR = MLPClassifier(solver='lbfgs', learning_rate='adaptive', alpha=1e-5, hidden_layer_sizes=(), activation='logistic', random_state=1, max_iter=100, shuffle=False)
MlpClfLR.fit(x_train,y_train)

print (MlpClfLR.score(x_train, y_train))
print (MlpClfLR.score(x_test, y_test))

y_hat_tr_mlpLR = MlpClfLR.predict(x_train)
y_hat_ts_mlpLR = MlpClfLR.predict(x_test)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr_mlpLR)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts_mlpLR)) * 100))

#print (MlpClfLR.coefs_)

0.3358333333333333
0.31438127090301005
train accuracy: 97.37752525252526 %
test accuracy: 97.37171041518867 %


In [11]:
NgxNt = y_test.shape[1]
M = y_test.shape[0]
#print(confusion_matrix(y_test[:,0], y_hat_ts[:,1]))
print (y_test[:,0].shape)
TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts_mlpLR[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

(299,)
0.505844397148745 0.46787270700314176 0.01537954798824364 0.0109033478598696


In [21]:
print (MlpClfLR.score(x_train, y_train))
print (MlpClfLR.score(x_test, y_test))

varAccMTLR = np.zeros(NgxNt)
for i in range(M):
    for j in range(NgxNt):
        varAccMTLR[j] = varAccMTLR[j] + (np.abs(y_test[i,j] - y_hat_ts_mlpLR[i,j])/M*100)
        
for j in range(NgxNt):
    varAccMTLR[j] = 100 - varAccMTLR[j]
    
print(np.amin(varAccMTLR))
print(np.mean(varAccMTLR))
print(varAccMTLR)

0.3358333333333333
0.31438127090301005
73.57859531772573
97.37171041518867
[100.         100.          92.30769231  92.30769231 100.
 100.          95.65217391  96.98996656 100.         100.
 100.         100.         100.         100.         100.
 100.         100.         100.          99.66555184 100.
  93.64548495  91.97324415 100.         100.         100.
 100.         100.         100.         100.         100.
  97.32441472  94.98327759  94.98327759 100.         100.
  92.30769231  91.97324415 100.          99.66555184  95.65217391
  96.98996656  99.66555184 100.          99.33110368 100.
 100.         100.         100.          99.66555184 100.
  99.33110368  99.33110368 100.          95.65217391  92.64214047
 100.         100.         100.         100.         100.
 100.         100.         100.          97.99331104  96.32107023
  94.98327759 100.          99.66555184  92.30769231  91.97324415
 100.         100.          95.65217391  96.98996656  94.64882943
  96.32107023  

In [39]:
weights = MlpClfLR.coefs_
weights = np.array(weights)
print (weights[0,:,2])

[-3.73088198e-02 -1.68712452e-02  3.10281591e-02 -2.80775485e-02
  5.22215685e-03 -3.69203811e-02  3.03330900e-02  3.84994673e-02
  8.80029628e-03 -1.03732521e-02 -2.87372597e-02  2.57067627e-02
  1.18795764e-02  3.22281786e-02  2.63976405e-02 -1.55935926e-02
 -1.69384037e-02  2.69181368e-02  2.42980920e-02  1.43524238e-02
  2.23006285e-02 -2.57153978e-02 -7.93596519e-03 -5.63174346e-03
  9.68583235e-03  3.56278140e-02 -7.38989829e-03 -1.96811187e-02
 -1.78517766e-02  1.39648832e-02 -2.74043777e-02  1.75356732e-02
 -1.65612355e-02 -3.37640517e-02  1.99418721e-02 -1.87158789e-02
  3.12468851e-02  3.41197073e-02 -1.62933894e-02  3.15479075e-02
  3.26927011e-02 -1.10721950e-02  3.48341202e-02  3.80471257e-02
 -1.91859888e-02  4.17887319e-03 -2.37263757e-02  5.59436360e-03
 -1.79512003e-02 -7.27885711e-04  1.06638476e-02  1.94472390e-02
  3.21063120e-02 -2.97314250e-02 -1.00929200e-03  1.83778668e-02
  3.46533234e-02  1.07638459e-03  8.78116823e-03 -3.12840358e-02
  3.57856902e-02  1.91573

In [40]:
from sklearn.linear_model import LogisticRegression

TrainScore = []
TrainHardScore = []
TestScore = []
TestHardScore = []
NgxNt = y_train.shape[1]
m_Tr = x_train.shape[0]
m_Ts = x_test.shape[0]
logreg2 = {}

Y_hat_tr_skln_pred2 = np.zeros((m_Tr,NgxNt))
Y_hat_ts_skln_pred2 = np.zeros((m_Ts,NgxNt))
listj = np.zeros(NgxNt)
start = timer()
j = 0
for i in range(NgxNt):
    #print ("Ng*Nt: ",i)   
    
    if set([0,1]).issubset(set(y_train[:,i])):      
        listj[i] = j
        logreg2[j] = LogisticRegression(random_state=1, solver='liblinear', max_iter = 1)
        logreg2[j].fit(x_train,y_train[:,i]) #original shape before transposed
        Y_hat_tr_skln_pred2[:,i] = logreg2[j].predict(x_train)
        TrainScore.append(logreg2[j].score(x_train,y_train[:,i]))
        Y_hat_ts_skln_pred2[:,i] = logreg2[j].predict(x_test)
        TestScore.append(logreg2[j].score(x_test,y_test[:,i]))
        j = j + 1
    else:
        
        if set([0]).issubset(set(y_train[:,i])):
            Y_hat_tr_skln_pred2[:,i] = np.zeros(m_Tr)
        else:
            Y_hat_tr_skln_pred2[:,i] = np.ones(m_Tr)
        
        if set([0]).issubset(set(y_test[:,i])):
            Y_hat_ts_skln_pred2[:,i] = np.zeros(m_Ts)
        else:
            Y_hat_ts_skln_pred2[:,i] = np.ones(m_Ts)
            
        #TrainScore.append(1)
        #TestScore.append(1)
end = timer()
print("training time: ",end - start)
print ("-------------------------------------------------------")

print("training Score: ", np.mean(TrainScore))
print("testing Score: ", np.mean(TestScore))

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - Y_hat_tr_skln_pred2)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - Y_hat_ts_skln_pred2)) * 100))

#print (logreg2[0].coefs_)

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\aramesh4\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the numbe

training time:  5.3711505000001125
-------------------------------------------------------
training Score:  0.9456627906976744
testing Score:  0.9445049389437661
train accuracy: 97.04987373737374 %
test accuracy: 96.9848991588122 %


In [41]:
print(listj)

[  0.   0.   1.   2.   0.   0.   3.   4.   0.   0.   0.   0.   0.   0.
   0.   0.   5.   0.   0.   0.   6.   7.   0.   0.   0.   0.   0.   0.
   0.   0.   8.   9.  10.   0.   0.  11.  12.   0.   0.  13.  14.  15.
  16.  17.   0.   0.   0.   0.  18.  19.  20.  21.  22.  23.  24.   0.
   0.   0.   0.   0.   0.   0.   0.  25.  26.  27.  28.  29.  30.  31.
   0.  32.  33.  34.  35.  36.  37.   0.   0.   0.   0.  38.  39.  40.
  41.  42.  43.  44.   0.   0.   0.   0.   0.   0.   0.   0.  45.  46.
  47.   0.  48.  49.  50.   0.   0.  51.  52.  53.  54.  55.   0.   0.
   0.   0.  56.  57.  58.  59.  60.  61.  62.   0.   0.   0.   0.   0.
   0.   0.   0.  63.  64.  65.   0.   0.  66.  67.   0.   0.  68.  69.
  70.  71.  72.  73.   0.  74.   0.  75.  76.  77.  78.  79.  80.  81.
   0.   0.   0.   0.   0.   0.   0.   0.  82.  83.  84.   0.  85.  86.
  87.   0.   0.  88.  89.  90.  91.  92.  93.  94.  95.   0.  96.  97.
  98.  99. 100. 101. 102.   0.   0.   0.   0.   0.   0.   0.   0. 103.
 104. 

In [42]:
weightsLR = logreg2[2].coef_
weightsLR = np.array(weightsLR)
print (weightsLR.shape)
print (weightsLR)

(1, 576)
[[0.00336527 0.00300674 0.00280698 0.0047676  0.00221003 0.00461275
  0.00193495 0.00263715 0.00545717 0.00311201 0.         0.
  0.00404549 0.00284444 0.00490473 0.00311352 0.         0.00503404
  0.00280457 0.00202649 0.         0.         0.         0.
  0.00339919 0.00307324 0.00283259 0.00484632 0.00223826 0.00465131
  0.00195115 0.00267833 0.00550966 0.00316014 0.         0.
  0.00410232 0.00288683 0.00498609 0.00315196 0.         0.00510448
  0.00282223 0.00204683 0.         0.         0.         0.
  0.00354433 0.00319147 0.00296399 0.00507448 0.00233222 0.00486647
  0.00205089 0.00279127 0.00578382 0.00328964 0.         0.
  0.00426218 0.00302726 0.00521318 0.00329328 0.         0.00536081
  0.00295682 0.00214433 0.         0.         0.         0.
  0.00374746 0.0033679  0.00313143 0.0053539  0.00246175 0.00511287
  0.00215176 0.00295357 0.00609997 0.00348074 0.         0.
  0.00451027 0.00319124 0.00544608 0.00348193 0.         0.00560521
  0.00312531 0.00225593 0. 

430
